In [68]:
from dotenv import load_dotenv
from rich import print
load_dotenv('./.env')
import json
import openai
import os
from dspy import (
    OpenAI,
    OllamaLocal,
    HFModel,
    Predict,
    Signature,
    ChainOfThought,
    Retrieve,
    settings,
    predict,
    context
)
import warnings
warnings.filterwarnings('ignore')

In [69]:
openai.api_key= os.environ['OPENAI_API_KEY']

In [70]:
turbo = OpenAI(model='gpt-3.5-turbo')
settings.configure(lm=turbo)

In [75]:
turbo("what is a cat?")

['A cat is a small domesticated carnivorous mammal with soft fur, a short snout, and retractable claws. Cats are known for their agility, independence, and playful nature. They are popular pets and are often kept for companionship and as rodent control.']

In [71]:
signatures={"QuestionAnswering":"question -> answer",
            "SentimentClassification":"sentence -> sentiment",
            "Summarization":["document -> summary",
                             "text -> gist",
                             "long_context -> tldr"],
            "RAG":"context, question -> answer",
            "Multi-Choice":"question, choices -> reasoning, selection"}

In [72]:
signatures['QuestionAnswering']

'question -> answer'

In [73]:
# Predict class uses a signature of this form 
# only a single Predict class is required
qa_pred = Predict('question -> answer')


In [74]:
print(qa_pred)

Predict(StringSignature(question -> answer
    instructions='Given the fields `question`, produce the fields `answer`.'
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 
'Question:', 'desc': '${question}'})
    answer = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 
'Answer:', 'desc': '${answer}'})
))

In [38]:
sum_cot = ChainOfThought('document -> summary')

In [39]:
print(sum_cot)

self = Predict(StringSignature(document -> rationale, summary
    instructions='Given the fields `document`, produce the fields `summary`.'
    document = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 
'Document:', 'desc': '${document}'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by 
step in order to", 'desc': '${produce the summary}. We ...', '__dspy_field_type': 'output'})
    summary = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 
'Summary:', 'desc': '${summary}'})
))

In [40]:
from dspy import(
    InputField,
    OutputField
)

class MultiClass(Signature):
    """Classift the given data into Address, Humans Name, Location, Building, Amount"""
    sentence = InputField(desc="data to be specified")
    data_type = OutputField(desc="falls in one of categories")



In [41]:
pred_class = Predict(MultiClass)
print(pred_class)

Predict(MultiClass(sentence -> data_type
    instructions='Classift the given data into Address, Humans Name, Location, Building, Amount'
    sentence = Field(annotation=str required=True json_schema_extra={'desc': 'data to be specified', 
'__dspy_field_type': 'input', 'prefix': 'Sentence:'})
    data_type = Field(annotation=str required=True json_schema_extra={'desc': 'falls in one of categories', 
'__dspy_field_type': 'output', 'prefix': 'Data Type:'})
))

In [42]:
get_json_data = Predict('required_data -> json_output')

prompt="provide me one eg of Address, Location, Human Name, building name and amount"

with context(lm=turbo):
    response = get_json_data(required_data=prompt)
    print("Response by OPENAI: ", response.json_output)

Response by OPENAI:  {
  "Address": "123 Main Street",
  "Location": "New York City",
  "Human Name": "John Doe",
  "Building Name": "Empire State Building",
  "Amount": "$100"
}

In [43]:
data= json.loads(response.json_output)

# Program of Thought


In [44]:
from dspy import ProgramOfThought

In [45]:
generate_answer_signature= Signature("question -> answer")

#pass the sig to ProgramOfThought Module
pot = ProgramOfThought(generate_answer_signature)

#call the program of thought Module on a prompt
question='Hritik has 5 apples. He buys 3 more apples. How many apples does he have now?'
result = pot(question=question)

print(f"Question:{question}")
print(f"Answer:{result}")


Question:Hritik has 5 apples. He buys 3 more apples. How many apples does he have now?

Answer:Prediction(
    rationale='produce the answer. We first define the initial number of apples Hritik has as 5 and the number of 
apples he buys as 3. By adding these two values together, we can calculate the total number of apples Hritik has 
now.',
    answer='Hritik has 8 apples now.'
)

In [46]:
turbo.inspect_history(n=6)




Given the fields `required_data`, produce the fields `json_output`.

---

Follow the following format.

Required Data: ${required_data}
Json Output: ${json_output}

---

Required Data: provide me one eg of Address, Location, Human Name, building name and amount
Json Output: {
  "Address": "123 Main Street",
  "Location": "New York City",
  "Human Name": "John Doe",
  "Building Name": "Empire State Building",
  "Amount": "$100"
}





You will be given `question` and you will respond with `answer`.
Generating executable Python code that programmatically computes the correct `answer`.
After you're done with the computation, make sure the last line in your code evaluates to the correct value for `answer`.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the generated_code}. We ...
Code: python code that answers the question

---

Question: Hritik has 5 apples. He buys 3 more apples. How many apples does he have now?
Rea

'\n\n\nGiven the fields `required_data`, produce the fields `json_output`.\n\n---\n\nFollow the following format.\n\nRequired Data: ${required_data}\nJson Output: ${json_output}\n\n---\n\nRequired Data: provide me one eg of Address, Location, Human Name, building name and amount\nJson Output:\x1b {\n  "Address": "123 Main Street",\n  "Location": "New York City",\n  "Human Name": "John Doe",\n  "Building Name": "Empire State Building",\n  "Amount": "$100"\n}\x1b\n\n\n\n\n\nYou will be given `question` and you will respond with `answer`.\nGenerating executable Python code that programmatically computes the correct `answer`.\nAfter you\'re done with the computation, make sure the last line in your code evaluates to the correct value for `answer`.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nReasoning: Let\'s think step by step in order to ${produce the generated_code}. We ...\nCode: python code that answers the question\n\n---\n\nQuestion: Hritik has 5 apples. He buys 

# Retrieve

In [47]:
# updating the LLM 

from dspy import ColBERTv2

colbert_wiki_rm=ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')
settings.configure(lm=turbo, rm=colbert_wiki_rm)

In [48]:
query ='When was the first fifa world cup held'

retrieve = Retrieve(k=2)

topk = retrieve(query).passages

for i, passage in enumerate(topk):
    print(f"Passage {i+1}: {passage} \n")



Passage 1: History of the FIFA World Cup | The FIFA World Cup was first held in 1930, when FIFA president Jules 
Rimet decided to stage an international football tournament. The inaugural edition, held in 1930, was contested as 
a final tournament of only thirteen teams invited by the organization. Since then, the World Cup has experienced 
successive expansions and format remodeling to its current 32-team final tournament preceded by a two-year 
qualifying process, involving over 200 teams from around the world.

Passage 2: 1950 FIFA World Cup | The 1950 FIFA World Cup, held in Brazil from 24 June to 16 July 1950, was the 
fourth FIFA World Cup. It was the first World Cup since 1938, the planned 1942 and 1946 competitions having been 
cancelled owing to World War II. It was won by Uruguay, who had won the inaugural competition in 1930, clinching 
the cup by beating the hosts Brazil 2–1 in the deciding match of the four-team final group (this was the only 
tournament not decided by a one-match final). It was also the first tournament where the trophy was referred to as 
the Jules Rimet Cup, to mark the 25th anniversary of Jules Rimet's presidency of FIFA.

In [49]:
raw_retrieval = retrieve(query)
print(raw_retrieval)

Prediction(
    passages=['History of the FIFA World Cup | The FIFA World Cup was first held in 1930, when FIFA president Jules
Rimet decided to stage an international football tournament. The inaugural edition, held in 1930, was contested as 
a final tournament of only thirteen teams invited by the organization. Since then, the World Cup has experienced 
successive expansions and format remodeling to its current 32-team final tournament preceded by a two-year 
qualifying process, involving over 200 teams from around the world.', "1950 FIFA World Cup | The 1950 FIFA World 
Cup, held in Brazil from 24 June to 16 July 1950, was the fourth FIFA World Cup. It was the first World Cup since 
1938, the planned 1942 and 1946 competitions having been cancelled owing to World War II. It was won by Uruguay, 
who had won the inaugural competition in 1930, clinching the cup by beating the hosts Brazil 2–1 in the deciding 
match of the four-team final group (this was the only tournament not decided by a one-match final). It was also the
first tournament where the trophy was referred to as the Jules Rimet Cup, to mark the 25th anniversary of Jules 
Rimet's presidency of FIFA."]
)

In [50]:
retrieval_qa = Predict("question, context -> answer")

output = retrieval_qa(question=query, context=raw_retrieval.passages)

In [51]:
print(output)

Prediction(
    answer='Question: When was the first FIFA World Cup held?\nContext: The FIFA World Cup was first held in 1930, 
when FIFA president Jules Rimet decided to stage an international football tournament.\nAnswer: 1930'
)

In [52]:
turbo.inspect_history(n=10)




Given the fields `required_data`, produce the fields `json_output`.

---

Follow the following format.

Required Data: ${required_data}
Json Output: ${json_output}

---

Required Data: provide me one eg of Address, Location, Human Name, building name and amount
Json Output: {
  "Address": "123 Main Street",
  "Location": "New York City",
  "Human Name": "John Doe",
  "Building Name": "Empire State Building",
  "Amount": "$100"
}





You will be given `question` and you will respond with `answer`.
Generating executable Python code that programmatically computes the correct `answer`.
After you're done with the computation, make sure the last line in your code evaluates to the correct value for `answer`.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the generated_code}. We ...
Code: python code that answers the question

---

Question: Hritik has 5 apples. He buys 3 more apples. How many apples does he have now?
Rea

'\n\n\nGiven the fields `required_data`, produce the fields `json_output`.\n\n---\n\nFollow the following format.\n\nRequired Data: ${required_data}\nJson Output: ${json_output}\n\n---\n\nRequired Data: provide me one eg of Address, Location, Human Name, building name and amount\nJson Output:\x1b {\n  "Address": "123 Main Street",\n  "Location": "New York City",\n  "Human Name": "John Doe",\n  "Building Name": "Empire State Building",\n  "Amount": "$100"\n}\x1b\n\n\n\n\n\nYou will be given `question` and you will respond with `answer`.\nGenerating executable Python code that programmatically computes the correct `answer`.\nAfter you\'re done with the computation, make sure the last line in your code evaluates to the correct value for `answer`.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nReasoning: Let\'s think step by step in order to ${produce the generated_code}. We ...\nCode: python code that answers the question\n\n---\n\nQuestion: Hritik has 5 apples. He buys 

# DsPY Data Code Execution

In [53]:
from dspy import Example

example1 = Example(question="where is the moon?",
                   answer='In Space',
                   hint="Check Science Book",
                   book_name="lost in space")
example1


Example({'question': 'where is the moon?', 'answer': 'In Space', 'hint': 'Check Science Book', 'book_name': 'lost in space'}) (input_keys=None)

In [54]:
example1.with_inputs("question")

Example({'question': 'where is the moon?', 'answer': 'In Space', 'hint': 'Check Science Book', 'book_name': 'lost in space'}) (input_keys={'question'})

In [55]:
tset = [Example(doc='A Very Long theory Paper',
                summary='Summary of the paper').with_inputs('doc')]

In [56]:
tset[0].inputs()



Example({'doc': 'A Very Long theory Paper'}) (input_keys={'doc'})

In [57]:
tset[0].labels()

Example({'summary': 'Summary of the paper'}) (input_keys=None)

# DSPy Metrics


In [67]:
from dspy.datasets import (
    HotPotQA,
)
hpqa = HotPotQA(train_seed=1, train_size=20,
                eval_seed=2, dev_size=5,
                test_size=0)  # Removed timeout argument
hpqa

KeyboardInterrupt: 

In [58]:
from dspy.evaluate import(
    answer_exact_match,
    answer_exact_match_str,
    answer_passage_match
)

from dspy import(
    Predict,
    Prediction,
    Example,
    InputField,
    OutputField,
    Signature,
    OpenAI, 
    settings,
    context,
    ChainOfThought
)

In [59]:
def validate_ans(example, pred, trace=None):
    return example.answer.lower()== pred.answer.lower()

In [60]:
# Multiple Checks 
def validate_context_and_answer(example, pred, contexts, trace=None):
    # check the gold label and the predicted answer are the same
    answer_match = example.answer.lower() == pred.answer.lower()

    # check the predicted answer comes from one of the retrieved contexts
    context_match = any((pred.answer.lower() in c) for c in contexts)

    if trace is None: # if we're doing evaluation or optimization
        return (answer_match + context_match) / 2.0
    else: # if we're doing bootstrapping, i.e. self-generating good demonstrations of each step
        return answer_match and context_match

In [61]:
class Assess(Signature):
    """Assess the quality of a tweet along with the specific dimension"""

    assessed_text = InputField()
    assessment_question = InputField()
    assessment_answer = OutputField(desc="Yes or No")

In [62]:
gpt3T = OpenAI(model = 'gpt-3.5-turbo',
               max_tokens=512,
               model_type='chat')

settings.configure(lm=gpt3T)

model_pred = Predict('question -> answer')
asses_pred = Predict(Assess)

In [ ]:
def metric(gold, pred, trace=None):
    question, answer, gen_answer = gold.question, gold.answer, pred.answer

    engaging = "Is the assessed text self-contained, information? "
    correct = f"The text should answer `{question}` with `{answer}`. Does the assessed text contain this answer?"

    
    